In [ ]:
import tensorflow as tf
X = tf.constant([[1., 2.], [3., 4.]])
y = tf.constant([[1.], [2.]])
w = tf.Variable(initial_value=[[1.], [2.]])
b = tf.Variable(initial_value=1.)
with tf.GradientTape() as tape:
    L = tf.reduce_sum(tf.square(tf.matmul(X, w) + b - y))
w_grad, b_grad = tape.gradient(L, [w, b])        # 计算L(w, b)关于w, b的偏导数
print(L, w_grad, b_grad)

In [ ]:
#线性回归，numpy

import numpy as np

X_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype = np.float32)
y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype = np.float32)

X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())

a, b = 0, 0

num_epoch = 10000
learning_rate = 5e-4
for e in range(num_epoch):
    #手动计算梯度
    y_pred = a * X + b
    grad_a, grad_b = 2 * (y_pred - y).dot(X), 2 * (y_pred - y).sum()
    
    #更新参数
    a, b = a - learning_rate * grad_a, b - learning_rate * grad_b
    
print(a, b)

In [ ]:
import tensorflow as tf
#tf构建线性回归
X = tf.constant(X)
y = tf.constant(y)

a = tf.Variable(initial_value=0.)
b = tf.Variable(initial_value=0.)
variables = [a, b]

num_epoch = 10000
optimizer = tf.keras.optimizers.SGD(learning_rate=5e-4)
for e in range(num_epoch):
    #使用tf.GradientTape()记录损失函数的梯度信息
    with tf.GradientTape() as tape:
        y_pred = a * X + b
        loss = tf.reduce_sum(tf.square(y_pred-y))
    #tf自动计算损失函数关于自变量的梯度
    grads = tape.gradient(loss, variables)
    #tf自动根据梯度更新参数
    optimizer.apply_gradients(grads_and_vars = zip(grads, variables))

In [ ]:
#模型类编写线性模型
import tensorflow as tf
X = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
y = tf.constant([[10.0], [20.0]])

class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(
            units = 1,
            activation = None,
            kernel_initializer = tf.zeros_initializer(),
            bias_initializer = tf.zeros_initializer()
        )
        
    def call(self, input):
        output = self.dense(input)
        return output
    
#以下代码类似前节
model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01)
for i in  range(100):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.reduce_mean(tf.square(y_pred - y))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars = zip(grads, model.variables))
print(model.variables)

## <模型

In [1]:
import numpy as np
import tensorflow as tf

In [ ]:
#-------------------------------------------------------
#多层感知机
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units = 100, activation = tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units = 10)
        
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense1(x)
        x = self.dense2(x)
        output = tf.nn.softmax(x)
        return output

In [6]:
import tensorflow as tf
#卷积神经网络
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters = 32,                 #卷积层神经元数据
            kernel_size = [5, 5],         #感受野大小
            padding = 'same',             #padding策略
            activation = tf.nn.relu       #激活函数
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size = [2, 2], strides = 2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters = 64,
            kernel_size = [5, 5],
            padding = 'same',
            activation = tf.nn.relu
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size = [2, 2], strides = 2)
        self.flatten = tf.keras.layers.Reshape(target_shape = (7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units = 1024, activation = tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units = 10)
        
    def call(self, inputs):
        x = self.conv1(inputs)                  # [batch_size, 28, 28, 32]
        x = self.pool1(x)                       # [batch_size, 14, 14, 32]
        x = self.conv2(x)                       # [batch_size, 14, 14, 64]
        x = self.pool2(x)                       # [batch_size, 7, 7, 64]
        x = self.flatten(x)                     # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)                      # [batch_size, 1024]
        x = self.dense2(x)                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

In [10]:
#-------------------------------------------------------
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

#-------------------------------------------------------
#模型的训练
num_epochs = 5
batch_size = 50
learning_rate = 0.001

#此处切换模型
model = CNN()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

#-------------------------------------------------------
#从 DataLoader 中随机取一批训练数据；
#将这批数据送入模型，计算出模型的预测值；
#将模型预测值与真实值进行比较，计算损失函数（loss）。这里使用 tf.keras.losses 中的交叉熵函数作为损失函数；
#计算损失函数关于模型变量的导数；
#将求出的导数值传入优化器，使用优化器的 apply_gradients 方法更新模型参数以最小化损失函数（优化器的详细使用方法见 前章 ）。
#-------------------------------------------------------

num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true = y, y_pred = y_pred)
        loss = tf.reduce_mean(loss)
        if batch_index % 100 == 0:
            print("batch {:d} : loss {:f}".format(batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars = zip(grads, model.variables))
#-------------------------------------------------------

#-------------------------------------------------------
#模型的评估
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index : end_index])
    sparse_categorical_accuracy.update_state(y_true = data_loader.test_label[start_index : end_index], y_pred = y_pred)
    print("test accuracy: {:f}".format(sparse_categorical_accuracy.result()))


#-------------------------------------------------------

batch 0 : loss 2.305252
batch 100 : loss 0.188559
batch 200 : loss 0.117815
batch 300 : loss 0.100690
batch 400 : loss 0.005754
batch 500 : loss 0.008916
batch 600 : loss 0.156306
batch 700 : loss 0.045422
batch 800 : loss 0.088421
batch 900 : loss 0.006576
batch 1000 : loss 0.112557
batch 1100 : loss 0.002607
batch 1200 : loss 0.018561
batch 1300 : loss 0.017290
batch 1400 : loss 0.004588
batch 1500 : loss 0.006743
batch 1600 : loss 0.052868
batch 1700 : loss 0.001867
batch 1800 : loss 0.035089
batch 1900 : loss 0.000640
batch 2000 : loss 0.009406
batch 2100 : loss 0.002940
batch 2200 : loss 0.000591
batch 2300 : loss 0.004280
batch 2400 : loss 0.013685
batch 2500 : loss 0.003754
batch 2600 : loss 0.001094
batch 2700 : loss 0.004257
batch 2800 : loss 0.000329
batch 2900 : loss 0.007263
batch 3000 : loss 0.064822
batch 3100 : loss 0.003088
batch 3200 : loss 0.002815
batch 3300 : loss 0.001333
batch 3400 : loss 0.011974
batch 3500 : loss 0.000504
batch 3600 : loss 0.001722
batch 3700 : 

## 模型>

## <加载预训练模型

In [12]:
import tensorflow as tf
import tensorflow_datasets as tfds

num_epoch = 5
batch_size = 50
learning_rate = 0.001

dataset = tfds.load("tf_flowers", split = tfds.Split.TRAIN, as_supervised = True)
dataset = dataset.map(lambda img, label : 
                      (tf.image.resize(img, (224, 224)) / 255.0, label)).shuffle(1024).batch(batch_size)
model = tf.keras.applications.MobileNetV2(weights = None, classes = 5)
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
for e in range(num_epoch):
    for images, labels in dataset:
        with tf.GradientTape() as tape:
            labels_pred = model(images, training = True)
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_true = labels, y_pred = labels_pred)
            loss = tf.reduce_mean(loss)
            print("loss {:f}".format(loss.numpy()))
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(grads_and_vars = zip(grads, model.trainable_variables))
    print(labels_pred)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset tf_flowers downloaded and prepared to /Users/zsf/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
loss 1.659369
loss 1.626653
loss 1.854700
loss 1.719930
loss 1.521780
loss 1.702531
loss 1.721047
loss 1.711823
loss 1.651302
loss 1.616478
loss 1.547647
loss 1.444698
loss 1.739620
loss 1.652479
loss 1.615724
loss 1.598410
loss 1.423571
loss 1.544024
loss 1.305873
loss 1.612223
loss 1.638282
loss 1.650394
loss 1.553843
loss 1.487343
loss 1.695044
loss 1.420199
loss 1.326383
loss 1.581424
loss 1.572337
loss 1.338717
loss 1.322334
loss 1.496048
loss 1.414958
loss 1.198110
loss 1.330192
loss 1.333858
loss 1.285416
loss 1.367896
loss 1.313110
loss 1.317251
loss 1.597445
loss 1.257945
loss 1.440295
loss 1.181853
loss 1.171274
loss 1.756959
loss 1.289944
loss 1.004329
loss 1.603307
loss 1.340686
loss 1.294433
loss 1.349244
loss 1.553134
loss 1.261371
loss 1.209425
loss 1.261326
loss 1.169861
loss 1.446637
loss 1.171175
loss 1.234749
loss 1.148795
loss 1.243

loss 0.839323
loss 0.883764
loss 0.629678
loss 0.695934
loss 0.756465
loss 0.734148
loss 0.640935
loss 0.644855
loss 0.870543
loss 0.828684
loss 0.627383
loss 1.158207
loss 0.522141
loss 0.826383
loss 0.803859
loss 0.582995
loss 0.646660
loss 0.589244
loss 0.669635
loss 0.701248
loss 0.819633
loss 0.590893
loss 0.675940
loss 0.627448
loss 0.883757
loss 0.612456
loss 0.893399
loss 0.694932
loss 0.545489
loss 0.482694
loss 0.612580
loss 0.515126
loss 0.521856
loss 0.951189
loss 0.511341
loss 0.811320
loss 0.512512
loss 0.889187
loss 0.559674
loss 0.949538
loss 0.709220
loss 0.650799
loss 0.899638
tf.Tensor(
[[1.74888343e-01 8.65133479e-02 2.63797790e-01 2.46691797e-02
  4.50131238e-01]
 [4.44245670e-04 1.26690924e-04 2.62346923e-01 3.92854010e-04
  7.36689210e-01]
 [1.93372682e-01 8.79162364e-03 5.77388844e-03 7.90941358e-01
  1.12039002e-03]
 [4.58042523e-05 1.60617928e-05 9.86060977e-01 2.89278716e-04
  1.35879675e-02]
 [3.23473290e-02 9.17320013e-01 2.06714645e-02 2.22334843e-02
  7.4